In [1]:
import joblib

# Load the trained model
rf_classifier = joblib.load('rf_classifier.pkl')

In [2]:
import math

def get_angle(point1, point2, point3):
    """ Calculate angle between two lines """
    if(point1==(0,0) or point2==(0,0) or point3==(0,0)):
        return 0
    numerator = point2[1] * (point1[0] - point3[0]) + point1[1] * \
                (point3[0] - point2[0]) + point3[1] * (point2[0] - point1[0])
    denominator = (point2[0] - point1[0]) * (point1[0] - point3[0]) + \
                (point2[1] - point1[1]) * (point1[1] - point3[1])

    try:
        ang = math.atan(numerator/denominator)

        ang = ang * 180 / math.pi
        if ang < 0:
            ang = 180 + ang
        return ang
    except:
        return 90.0

In [3]:
temp_dir = 'temp_frames'

In [4]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import cv2
import os

# Function to perform pose estimation and get angles for an image
def get_angles():

    # Load the temporary frame
    temp_frame_path = os.path.join(temp_dir, 'temp_frame.jpg')

    # Create a HandLandmarker object.
    base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
    options = vision.HandLandmarkerOptions(base_options=base_options, num_hands=2)
    detector = vision.HandLandmarker.create_from_options(options)

        # Create mp.Image from file
    image = mp.Image.create_from_file(temp_frame_path)

    # Process image
    results = detector.detect(image)

    hand_landmarks_list = results.hand_landmarks

    if hand_landmarks_list:
        # Your existing code for extracting angles
        image_landmarks = {}
        for idx in range(len(hand_landmarks_list)):
            hand_landmarks = hand_landmarks_list[idx]

            hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
            hand_landmarks_proto.landmark.extend([
                landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
            ])

            for i, data_point in enumerate(hand_landmarks_proto.landmark):
                image_landmarks[i] = (data_point.x, data_point.y)

        # Calculate angles between hand landmarks
        angle0 = get_angle(image_landmarks[4], image_landmarks[3], image_landmarks[2])
        angle1 = get_angle(image_landmarks[3], image_landmarks[2], image_landmarks[1])
        angle2 = get_angle(image_landmarks[2], image_landmarks[1], image_landmarks[0])
        angle3 = get_angle(image_landmarks[1], image_landmarks[0], image_landmarks[5])
        angle4 = get_angle(image_landmarks[8], image_landmarks[7], image_landmarks[6])
        angle5 = get_angle(image_landmarks[7], image_landmarks[6], image_landmarks[5])
        angle6 = get_angle(image_landmarks[6], image_landmarks[5], image_landmarks[0])
        angle7 = get_angle(image_landmarks[12], image_landmarks[11], image_landmarks[10])
        angle8 = get_angle(image_landmarks[11], image_landmarks[10], image_landmarks[9])
        angle9 = get_angle(image_landmarks[10], image_landmarks[9], image_landmarks[13])
        angle10 = get_angle(image_landmarks[16], image_landmarks[15], image_landmarks[14])
        angle11 = get_angle(image_landmarks[15], image_landmarks[14], image_landmarks[13])
        angle12 = get_angle(image_landmarks[14], image_landmarks[13], image_landmarks[17])
        angle13 = get_angle(image_landmarks[20], image_landmarks[19], image_landmarks[18])
        angle14 = get_angle(image_landmarks[19], image_landmarks[18], image_landmarks[17])
        angle15 = get_angle(image_landmarks[18], image_landmarks[17], image_landmarks[0])

        # Return angles as a dictionary
        return [angle0, angle1, angle2, angle3, angle4, angle5, angle6, angle7, angle8, angle9,
                                            angle10, angle11, angle12, angle13, angle14, angle15]

    else:
        # Return None if no hand landmarks are detected
        return None

In [ ]:
import cv2
import os

# Open the webcam
cap = cv2.VideoCapture(0)

# Temporary directory to save the frames
os.makedirs(temp_dir, exist_ok=True)

while True:
    # Capture a frame
    ret, frame = cap.read()

    # Save the frame temporarily
    cv2.imwrite(os.path.join(temp_dir, 'temp_frame.jpg'), frame)

    # Get angles from the current frame
    angles = get_angles()

    angles_array = np.array(angles, dtype=np.float64)

    # Check if angles array contains NaN
    if np.isnan(angles_array).any():
        print("No hand detected. Skipping prediction.")
        continue
    
    # Convert the list of lists into a NumPy array
    angles_array = np.array(angles)
    angles_array = angles_array.reshape(1, -1)

    # Make predictions using your model

    asl_category_mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'del': 26, 'space': 27}

    # Reverse mapping to get class names from predicted numbers
    asl_class_names = {v: k for k, v in asl_category_mapping.items()}

    # Make predictions using your model
    predictions = rf_classifier.predict(angles_array)

    # Map the predicted numbers to class names
    predicted_classes = [asl_class_names[prediction] for prediction in predictions]

    # Display predictions on the frame
    cv2.putText(frame, f'Predicted: {predicted_classes}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame with OpenCV
    cv2.imshow('ASL Prediction', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object
cap.release()
cv2.destroyAllWindows()
